In [1]:
import pandas as pd
import json

# 读取 JSONL 文件
df_text = pd.read_json('pretraining_text.jsonl', lines=True)
df_mm = pd.read_json('pretraining_mm.jsonl', lines=True)

# # 显示 DataFrame
# print(df)

,Instruction,Response
0,please find which nodes point to ripple_adder_...,"iSAOUT point to ripple_adder_32, ripple_adder_..."
1,"please find which nodes point to data_a node, ...","op1 point to data_a, data_a point to none."
2,"please find which nodes point to iSAOUT node, ...","No nodes point to iSAOUT,iSAOUT points to data..."
3,"please find which nodes point to led_0 node, a...","data_a point to led_0,led_0 points to out8,out..."
4,please find which nodes point to ASYNC_TO_SYNC...,"No nodes point to ASYNC_TO_SYNC, ASYNC_TO_SYNC..."
...,...,...
134995,"please find which nodes point to dff19 node, a...","No nodes point to dff19,dff19 points to bankti..."
134996,"please find which nodes point to select node, ...","No nodes point to select, select point to none."
134997,"please find which nodes point to INPUT_2 node,...","No nodes point to INPUT_2,INPUT_2 points to ou..."
134998,please find which nodes point to three_to_four...,No nodes point to three_to_four_wire_connectio...


In [2]:
num_lines = len(df_text)

In [3]:
template = "Given a submodules interconnection graph: \n<graph>\n, "
# text_graph = '\n{}\n,'.format(graph)

In [4]:
new_text = []
for i in range(num_lines):
# for i in range(3):
    new_line = []
    graph = "nodes: {}, \nedge_attrs: {}, \nconnectivity: {}".format(df_mm['nodes'][i], df_mm['edge_attrs'][i], df_mm['connectivity'][i])
    text_graph = "\n{}\n, ".format(graph)
    human_instruction = template + text_graph + df_text['Instruction'][i]
    human_instruction = {"from": "human", "value": human_instruction}
    gpt_response = df_text['Response'][i]
    gpt_response = {"from": "gpt", "value": gpt_response}
    new_line.append(human_instruction)
    new_line.append(gpt_response)
    new_line = {"conversations": new_line}
    new_text.append(new_line)
    # print(json.dumps(new_line))


In [5]:
with open('./embedding_text/conversations.json', 'w') as f:
    # for line in new_text:
    #     f.write(json.dumps(line) + '\n')
    json.dump(new_text, f)

In [6]:
new_graph_df = df_mm[['nodes', 'edge_attrs', 'connectivity']]
new_graph_df
new_graph_df.to_json('./embedding_text/graph.jsonl', orient='records', lines=True)

,nodes,edge_attrs,connectivity
0,"[{'id': 0, 'content': 'data_a'}, {'id': 1, 'co...",[],"[[1, 4], [5, 4]]"
1,"[{'id': 0, 'content': 'data_a'}, {'id': 1, 'co...",[],"[[4, 4, 4], [0, 4, 1]]"
2,"[{'id': 0, 'content': 'data_a'}, {'id': 1, 'co...",[],"[[2, 1, 0, 1], [4, 0, 5, 3]]"
3,"[{'id': 0, 'content': 'data_a'}, {'id': 1, 'co...",[],"[[2, 3, 3, 0, 3], [4, 2, 2, 3, 2]]"
4,"[{'id': 0, 'content': 'vpb'}, {'id': 1, 'conte...",[],"[[3, 3], [1, 5]]"
...,...,...,...
134995,"[{'id': 0, 'content': 'INPUT_2'}, {'id': 1, 'c...",[],"[[8, 11, 9, 0, 5, 4, 4, 2], [0, 10, 11, 11, 12..."
134996,"[{'id': 0, 'content': 'INPUT_2'}, {'id': 1, 'c...",[],"[[2, 8, 10, 0, 1, 4, 12, 0, 9], [3, 6, 7, 12, ..."
134997,"[{'id': 0, 'content': 'INPUT_2'}, {'id': 1, 'c...",[],"[[0, 4, 9, 1, 4, 0, 1, 12, 4, 1], [4, 2, 4, 9,..."
134998,"[{'id': 0, 'content': 'INPUT_2'}, {'id': 1, 'c...",[],"[[5, 1, 5, 9, 3, 12, 6, 9, 6, 7, 6], [2, 3, 11..."
